# CSCM72 Coursework 1: Python Optimisers

# Section 1: Function Implementation & Testing

In [2]:
import numpy as np
import math

In [3]:
# Constants to improve readability
HEIGHT = 0     #x1
LENGTH = 1     #x2
THICKNESS = 2  #x3
BREADTH = 3    #x4

## Constraint Function Execution Tracking
The number of executions of each constraint function is tracked with a global variable. 
A function is included that prints the state of all these variables, to make checking them easier. 

In [27]:
# Counter Variables to track the number of time Constraint Functions have been executed
g1_counter = 0
g2_counter = 0
g3_counter = 0
g4_counter = 0

# Prints how many times each constraint function has been executed
def print_execution_trackers():
    global g1_counter
    global g2_counter
    global g3_counter
    global g4_counter
    print(
        "Constraint function Execution counts: \ng1: {}, g2: {}, g3: {}, g4: {}".format(g1_counter, g2_counter, g3_counter, g4_counter)
    )

## Function Implementations

This subsection defines the function implementations for the objective function and each of the constraint functions. 

### Objective Function *f(x)*:

In [5]:
def objective_calc(design):
    return ( 1.10471 * (design[HEIGHT] ** 2) * design[LENGTH]) + (0.04811 * design[THICKNESS] * design[BREADTH] * (14.0 + design[LENGTH]) )

### Shear Stress *g<sub>1</sub>(x)*:

In [6]:
def shear_stress(design):
    # Calculates the first derivative
    tau1 = 6000 / (math.sqrt(2) * design[HEIGHT] * design[LENGTH])

    # Calculates the second derivative
    tau2 = (6000 * (14 + 0.5 * design[LENGTH]) * math.sqrt(0.25 * (design[LENGTH]**2 + (design[HEIGHT] + design[THICKNESS])**2 ) ) )  \
                        / ( 2 * (0.707 * design[HEIGHT] * design[LENGTH] * ( ( (design[LENGTH] ** 2) / 12) + 0.25 * (design[HEIGHT] + design[THICKNESS])**2 ) ) )
    
    # Calculates the overall shear stress of the design
    tau = math.sqrt( (tau1**2 + tau2**2) + ( (design[LENGTH] * tau1 * tau2) \
           / (math.sqrt(0.25*(design[LENGTH]**2 + (design[HEIGHT] + design[THICKNESS])**2))) ))

    global g1_counter
    g1_counter += 1
    return 13600 - tau

### Normal Stress *g<sub>2</sub>(x)*:

In [7]:
def normal_stress(design):
    # Calculates normal stress of the design
    sigma = 504000 / (design[THICKNESS]**2 * design[BREADTH])

    global g2_counter
    g2_counter += 1
    return (30000 - sigma)

### Practicality *g<sub>3</sub>(x)*:

In [8]:
def practicality(design):
    global g3_counter
    g3_counter += 1
    return (design[BREADTH] - design[HEIGHT])

### Buckling Load *g<sub>4</sub>(x)*:

In [9]:
def buckling_load(design):
    rho = 64746.022 * (1 - 0.0282346 * design[THICKNESS]) * design[THICKNESS] * design[BREADTH]**3

    global g4_counter
    g4_counter += 1
    return (rho - 6000)

## Function Validation:

This section checks the validity of the constraint functions, using the example input design from the specification.

In [10]:
x = np.array([1.05, 3.15, 4.43, 7.87])

print("Objective Function Output: ", objective_calc(x))
print("First constraint function output: ", shear_stress(x))
print("Second constraint function output: ", normal_stress(x))
print("Third constraint function output: ", practicality(x))
print("Fourth constraint function output: ", buckling_load(x))

Objective Function Output:  32.6024179859
First constraint function output:  5308.848564674312
Second constraint function output:  26736.764990548952
Third constraint function output:  6.82
Fourth constraint function output:  122317448.61430933


In [11]:
print("Execution counters: g1 = {}, g2 = {}, g3 = {}, g4 = {}".format(g1_counter, g2_counter, g3_counter, g4_counter))

Execution counters: g1 = 1, g2 = 1, g3 = 1, g4 = 1


# Section 2: Random Search

In [24]:
# This function performs random search optimisation using a given seed for number generation, and a given number of samples
def random_search(seed, repeats):
    # Initialises algorithm variables
    best_design = None
    objective_minimum = float('inf')

    # Creates local random generator to isolate from seed changes in any other code
    rng = np.random.default_rng(seed)

    for i in range(repeats):
        x1 = rng.uniform(0.125, 5.0)
        x2 = rng.uniform(0.125, 5.0)
        x3 = rng.uniform(0.1, 10.0)
        x4 = rng.uniform(0.1, 10.0)

        curr_design = np.array([x1, x2, x3, x4])

        if (shear_stress(curr_design) >= 0) and (normal_stress(curr_design) >= 0) and (practicality(curr_design) >= 0) and (buckling_load(curr_design) >= 0):
            curr_result = objective_calc(curr_design)
            if curr_result < objective_minimum:
                objective_minimum = curr_result
                best_design = curr_design
    print("Best Design: {}, with Objective Value: {}".format(best_design, objective_minimum))

In [28]:
random_search(21, 10000)
random_search(18002, 10000)

print_execution_trackers()

Best Design: [0.73546113 2.47576191 5.83365327 0.74235623], with Objective Value: 4.9120533830155555
Best Design: [0.52367905 2.67381647 7.43394281 0.56708869], with Objective Value: 4.191789951308734
Constraint function Execution counts: 
g1: 20000, g2: 15716, g3: 12749, g4: 9816
